In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
# products_sheet_id = "1FB2bbvWE2nsf49KO_DkWzbL2hm9r4G8j"
# products_list = "Tablib%20Dataset"
# url = f"https://docs.google.com/spreadsheets/d/{products_sheet_id}/gviz/tq?tqx=out:csv&sheet={products_list}"

In [2]:
# convert csv datasets into dataframes

pdt_df = pd.read_csv('full_products_list.csv')
batches_df = pd.read_csv('batches_list.csv')
recipe_df = pd.read_csv('recipe_list.csv')

In [3]:
# import excel file from an absolute path with pandas

xls = pd.ExcelFile(r'C:\Users\NDA\Desktop\EVE\TMCG\documents\duplicate_products.xlsx')

In [4]:
# convert xls datasets on sheet names into dataframes

dp_list = pd.read_excel(xls, 'copy_final_list_duplicates')
unique_pdts_df = pd.read_excel(xls, 'unique final list')

In [5]:
# drop unwanted columns from the duplicates list dataframe

new_dp_list = dp_list.drop(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 8',
                           'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
                           'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
                           'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23'], axis=1)

In [6]:
# cleanup data by renaming pandas series with more meaningfull names

new_dp_list = new_dp_list.rename(columns={'Eva': 'PRODUCT ID', 'Unnamed: 6': 'GENERIC NAME', 'Unnamed: 7': 'BRAND NAME'})

In [7]:
# drop null values based on rows

new_dp_list = new_dp_list.dropna(axis='rows')

In [ ]:
# unique_pdts_df

In [ ]:
# check to see if dataset is now clean 

new_dp_list.tail()

In [8]:
# remove the inverse labels

new_dp_list = new_dp_list.drop([new_dp_list.index[-1]])

In [ ]:
# check to see whether id has numerical values
# re-check after running next cell

new_dp_list.info()

In [9]:
# convert product id into numeric 

new_dp_list['PRODUCT ID'] = pd.to_numeric(new_dp_list['PRODUCT ID'], errors='ignore')

In [ ]:
new_row = unique_pdts_df.iloc[14]
new_row

In [ ]:
new_dp_list = new_dp_list.append(new_row, ignore_index=True)

In [ ]:
new_dp_list.head()

In [ ]:
new_dp_list

In [ ]:
# check unique products dataframe

unique_pdts_df.columns

In [10]:
# drop unnecessary columns from unique lists

unique_pdts_df = unique_pdts_df.drop(['COUNTRY OF ORIGIN', 'FORMULATION', 'RETAIL PRICE PER SMALLEST UNIT', 
                                      'RETAIL PRICE PER SMALLEST UNIT.1', 'PACK SIZE', 'STRENGTH', 'eshop_id', 
                                      'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 
                                      'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 
                                      'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 
                                      'Unnamed: 25'], axis=1)

In [11]:
# change column name into product id for easy manipulation

unique_pdts_df = unique_pdts_df.rename(columns={'ID': 'PRODUCT ID'})

In [ ]:
# unique_pdts_df = unique_pdts_df.drop(labels=14, axis=0)

In [ ]:
# unique_pdts_df.reset_index(drop=True, inplace=True)

In [ ]:
unique_pdts_df

In [12]:
# give proper indices to the dataset

new_dp_list.reset_index(drop=True, inplace=True)

In [13]:
# create a new column on which we will compare our product ids
new_dp_list['NEW ID'] = ''

# create a new column on which we will compare our generic names
new_dp_list['new_generic_name'] = ''

# loop through the unique products dataframe using its index
for i in range(len(unique_pdts_df)):
    
   # loop through the list of duplicates using its index
    for j in range(len(new_dp_list)):
        
       # compare the two pandas series on generic names. Coerce them to lowercase for proper comparison
        if unique_pdts_df['GENERIC NAME'][i].lower() in new_dp_list['GENERIC NAME'][j].lower():
            
            # assign the new ids to the resulting product from unique products loop and do the same for generic name
            new_dp_list['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]
            new_dp_list['new_generic_name'][j] = unique_pdts_df['GENERIC NAME'][i]        

<ipython-input-13-459ad8c5262c>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dp_list['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]
<ipython-input-13-459ad8c5262c>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dp_list['new_generic_name'][j] = unique_pdts_df['GENERIC NAME'][i]


In [14]:
# subset the rows that were filled
filled = new_dp_list[new_dp_list['NEW ID'] != '']

# filled

In [15]:
# subset the unfilled rows
unfilled = new_dp_list[new_dp_list['NEW ID'] == '']

# unfilled

In [16]:
# re-index the dataframe because dropping out rows has messed up with its indexing

unfilled.reset_index(drop=True, inplace=True)

In [17]:
unique_pdts_df

,PRODUCT ID,GENERIC NAME,BRAND NAME
0,5302,Glyceryl trinitrate,accord 500mcg
1,3949,Betadine Antiseptic Solution,Betadine Antiseptic Solution
2,715,Montelukast,DENK AIR 10MG
3,4328,Diclofenac SR Tabs,Diclomol
4,830,Doxazosin,DOXAZOSIN 4MG UK
5,4392,Duo Cortexin Adult,Duo Cortexin Adult
6,295,"Dihdroartemisinine,Piperaquine",DUO-COTECXIN 40/320
7,4337,Diclofenac Injection,Dynapar AQ
8,5569,Dynapar Injection,DYNAPAR INJECTION
9,4403,E nat caps,E nat caps


In [ ]:
new_dp_list

In [ ]:
new_dp_list.head()

In [25]:
unique_dict = {k:[] for k in new_dp_list['PRODUCT ID']}

for i in range(len(unique_pdts_df)):
    
   # loop through the list of duplicates using its index
    for j in range(len(new_dp_list)):
        
       # compare the two pandas series on generic names. Coerce them to lowercase for proper comparison
        if unique_pdts_df['GENERIC NAME'][i].lower() in new_dp_list['GENERIC NAME'][j].lower():
            
            unique_dict[new_dp_list['PRODUCT ID'][j]].append(unique_pdts_df['PRODUCT ID'][i])
            
unique_dict

# loop through the unique products dataframe using its index
# for i in range(len(unique_pdts_df)):
# #     print(unique_dict)
    
#     for key, val in unique_dict.items():
# #         if unique_pdts_df['GENERIC NAME'][i].lower() in new_dp_list['Generic Name'][j].lower():
#         val.append(unique_pdts_df['PRODUCT ID'][i])
# #         print(val)
       

{3951: [3352],
 387: [715],
 4933: [715],
 4942: [715],
 3146: [],
 5312: [830],
 5416: [],
 3819: [],
 5304: [5303],
 5239: [],
 4685: [4686],
 4672: [],
 5444: [569],
 4896: [166],
 4888: [166],
 209: [166],
 5033: [5533],
 530: [],
 418: [],
 261: [],
 4477: [5134],
 4865: [4855],
 3915: [638],
 3904: [638],
 3304: [638],
 253: [],
 5234: [],
 3325: [3925],
 2725: [3925],
 3952: [3352, 4594],
 4595: [715],
 719: [696],
 4057: [4056],
 4132: [3532],
 5391: [],
 5390: [],
 5389: [],
 5031: [902],
 394: [902],
 902: [902],
 4031: [4029],
 5378: [5347],
 4729: [715],
 5370: [647],
 540: [],
 4868: [4867],
 5380: [],
 5379: [],
 3814: [2614],
 538: [173],
 636: [5152],
 5394: [4442]}

In [18]:
unique_dict = {k.lower():[] for k in new_dp_list['GENERIC NAME']}

for i in range(len(unique_pdts_df)):
    
   # loop through the list of duplicates using its index
    for j in range(len(new_dp_list)):
        
       # compare the two pandas series on generic names. Coerce them to lowercase for proper comparison
        if unique_pdts_df['GENERIC NAME'][i].lower() in new_dp_list['GENERIC NAME'][j].lower():
            
            unique_dict[new_dp_list['GENERIC NAME'][j].lower()].append(unique_pdts_df['PRODUCT ID'][i])
            
unique_dict

{'betadine mouthwash gargle': [3352, 3352],
 'montelukast': [715],
 'montelukast tabs': [715, 715],
 'diclofenac tabs': [],
 'doxazosin': [830],
 'levothyroxine': [],
 'amlodipine besilate,valsatan': [],
 'amlodipine/ valsartan': [5303],
 'face shields': [],
 'ketoprofen gel': [4686],
 'kabuuti herbal cough syrup': [],
 'lamotrigine': [569],
 'metronidazole gel': [166],
 'metronidazole susp': [166],
 'metronidazole': [166],
 'desmopressin': [5533],
 'loratidine': [],
 'fluoxetine caps': [5134],
 'metformin sr tabs': [4855],
 'azithromycin susp': [638, 638, 638],
 'desloratidine': [],
 'atorvastatin 20mg': [],
 'beechams max strength all in one caps': [3925, 3925],
 'hyoscine butylbromide syp': [4594],
 'levocetirizine+montelukast tab': [715],
 'clopidogrel': [696],
 'caustic pencil': [4056],
 'ciprofloxacin tabs': [3532],
 'diclofenac 12.5mg suppositories': [],
 'face mask': [902, 902, 902],
 'captopril tabs': [4029],
 'fluconazole': [5347],
 'levocetirizine&montelukast tabs': [715],
 

In [26]:
# loop through the unique products dataframe using its index
for i in range(len(unique_pdts_df)):
    
    # loop through the list of duplicates using its index
    for j in range(len(new_dp_list)):
        if len(unique_dict[new_dp_list['PRODUCT ID'][j]]) == 0:
        
            # extract string from the product
            str_a  = unique_pdts_df['GENERIC NAME'][i].lower()
            str_b = new_dp_list['GENERIC NAME'][j].lower()

            # assign ratio to both strings
            ratio = fuzz.partial_ratio(str_b, str_a)            

            # sent conditon for comparing the series
            if ratio >= 81:
                unique_dict[new_dp_list['PRODUCT ID'][j]].append(unique_pdts_df['PRODUCT ID'][i])
unique_dict

    #                 unique_dict[key] = str_b
    #                 unique_dict.append([str_a])
    #                 print(unique_dict)
    #                 unfilled['new_generic_name'][j] = unique_pdts_df['GENERIC NAME'][i]
    #                 unfilled['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]

{3951: [3352],
 387: [715],
 4933: [715],
 4942: [715],
 3146: [],
 5312: [830],
 5416: [4745],
 3819: [3816],
 5304: [5303],
 5239: [5044],
 4685: [4686],
 4672: [],
 5444: [569],
 4896: [166],
 4888: [166],
 209: [166],
 5033: [5533],
 530: [4786],
 418: [4786],
 261: [4786],
 4477: [5134],
 4865: [4855],
 3915: [638],
 3904: [638],
 3304: [638],
 253: [4786],
 5234: [5208],
 3325: [3925],
 2725: [3925],
 3952: [3352, 4594],
 4595: [715],
 719: [696],
 4057: [4056],
 4132: [3532],
 5391: [],
 5390: [],
 5389: [],
 5031: [902],
 394: [902],
 902: [902],
 4031: [4029],
 5378: [5347],
 4729: [715],
 5370: [647],
 540: [],
 4868: [4867],
 5380: [5337],
 5379: [5337],
 3814: [2614],
 538: [173],
 636: [5152],
 5394: [4442]}

In [19]:
# loop through the unique products dataframe using its index
for i in range(len(unique_pdts_df)):
    
    # loop through the list of duplicates using its index
    for j in range(len(new_dp_list)):
        if len(unique_dict[new_dp_list['GENERIC NAME'][j].lower()]) == 0:
        
            # extract string from the product
            str_a  = unique_pdts_df['GENERIC NAME'][i].lower()
            str_b = new_dp_list['GENERIC NAME'][j].lower()

            # assign ratio to both strings
            ratio = fuzz.partial_ratio(str_b, str_a)            

            # sent conditon for comparing the series
            if ratio >= 81:
                unique_dict[new_dp_list['GENERIC NAME'][j].lower()].append(unique_pdts_df['PRODUCT ID'][i])
unique_dict

{'betadine mouthwash gargle': [3352, 3352],
 'montelukast': [715],
 'montelukast tabs': [715, 715],
 'diclofenac tabs': [],
 'doxazosin': [830],
 'levothyroxine': [4745],
 'amlodipine besilate,valsatan': [3816],
 'amlodipine/ valsartan': [5303],
 'face shields': [5044],
 'ketoprofen gel': [4686],
 'kabuuti herbal cough syrup': [],
 'lamotrigine': [569],
 'metronidazole gel': [166],
 'metronidazole susp': [166],
 'metronidazole': [166],
 'desmopressin': [5533],
 'loratidine': [4786],
 'fluoxetine caps': [5134],
 'metformin sr tabs': [4855],
 'azithromycin susp': [638, 638, 638],
 'desloratidine': [4786],
 'atorvastatin 20mg': [5208],
 'beechams max strength all in one caps': [3925, 3925],
 'hyoscine butylbromide syp': [4594],
 'levocetirizine+montelukast tab': [715],
 'clopidogrel': [696],
 'caustic pencil': [4056],
 'ciprofloxacin tabs': [3532],
 'diclofenac 12.5mg suppositories': [],
 'face mask': [902, 902, 902],
 'captopril tabs': [4029],
 'fluconazole': [5347],
 'levocetirizine&mon

In [20]:
unique_dict['miconazole, clebetasol and gentamycin'].append(346)
unique_dict['diclofenac 12.5mg suppositories'].append(4334)
unique_dict['kabuuti herbal cough syrup'].append(5145)
unique_dict['diclofenac tabs'].append(4328)
unique_dict['amlodipine besilate,valsatan'].append(5303)

In [ ]:
unique_dict[540].append(346)
unique_dict[5391, 5390, 5391].append(4334)
unique_dict[4672].append(5145)
unique_dict[3146].append(4328)
unique_dict['amlodipine besilate,valsatan'].append(5303)

In [21]:
unique_dict

{'betadine mouthwash gargle': [3352, 3352],
 'montelukast': [715],
 'montelukast tabs': [715, 715],
 'diclofenac tabs': [4328],
 'doxazosin': [830],
 'levothyroxine': [4745],
 'amlodipine besilate,valsatan': [3816, 5303],
 'amlodipine/ valsartan': [5303],
 'face shields': [5044],
 'ketoprofen gel': [4686],
 'kabuuti herbal cough syrup': [5145],
 'lamotrigine': [569],
 'metronidazole gel': [166],
 'metronidazole susp': [166],
 'metronidazole': [166],
 'desmopressin': [5533],
 'loratidine': [4786],
 'fluoxetine caps': [5134],
 'metformin sr tabs': [4855],
 'azithromycin susp': [638, 638, 638],
 'desloratidine': [4786],
 'atorvastatin 20mg': [5208],
 'beechams max strength all in one caps': [3925, 3925],
 'hyoscine butylbromide syp': [4594],
 'levocetirizine+montelukast tab': [715],
 'clopidogrel': [696],
 'caustic pencil': [4056],
 'ciprofloxacin tabs': [3532],
 'diclofenac 12.5mg suppositories': [4334],
 'face mask': [902, 902, 902],
 'captopril tabs': [4029],
 'fluconazole': [5347],
 '

In [23]:
unique_dict['amlodipine besilate,valsatan'].pop(0)

3816

In [24]:
unique_dict

{'betadine mouthwash gargle': [3352, 3352],
 'montelukast': [715],
 'montelukast tabs': [715, 715],
 'diclofenac tabs': [4328],
 'doxazosin': [830],
 'levothyroxine': [4745],
 'amlodipine besilate,valsatan': [5303],
 'amlodipine/ valsartan': [5303],
 'face shields': [5044],
 'ketoprofen gel': [4686],
 'kabuuti herbal cough syrup': [5145],
 'lamotrigine': [569],
 'metronidazole gel': [166],
 'metronidazole susp': [166],
 'metronidazole': [166],
 'desmopressin': [5533],
 'loratidine': [4786],
 'fluoxetine caps': [5134],
 'metformin sr tabs': [4855],
 'azithromycin susp': [638, 638, 638],
 'desloratidine': [4786],
 'atorvastatin 20mg': [5208],
 'beechams max strength all in one caps': [3925, 3925],
 'hyoscine butylbromide syp': [4594],
 'levocetirizine+montelukast tab': [715],
 'clopidogrel': [696],
 'caustic pencil': [4056],
 'ciprofloxacin tabs': [3532],
 'diclofenac 12.5mg suppositories': [4334],
 'face mask': [902, 902, 902],
 'captopril tabs': [4029],
 'fluconazole': [5347],
 'levoce

In [ ]:
# loop through the unique products dataframe using its index
for i in range(len(unique_pdts_df)):

    # loop through the list of duplicates using its index
    for j in range(len(unfilled)):
        
        # extract string from the product
        str_a  = unique_pdts_df['GENERIC NAME'][i].lower()
        str_b = unfilled['GENERIC NAME'][j].lower()
        
        # assign ratio to both strings
        ratio = fuzz.partial_ratio(str_b, str_a)            
        
        # sent conditon for comparing the series
        if ratio >= 81:
            print(str_b, ':', str_a, ':', ratio)
            unfilled['new_generic_name'][j] = unique_pdts_df['GENERIC NAME'][i]
            unfilled['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]            
#                 unique_dict[key] = str_b
#                 unique_dict.append([str_a])
#                 print(unique_dict)
#                 unfilled['new_generic_name'][j] = unique_pdts_df['GENERIC NAME'][i]
#                 unfilled['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]

In [ ]:
unfilled

In [ ]:
dict_new = my_dict[str_b] == [str_a]
            print(dict_new)

In [ ]:
fuzz_filter(unique_pdts_df, unfilled).head()

In [ ]:
unfilled

In [ ]:
# combine the dataframes into one dataframe
combined_df = pd.concat([unfilled, filled], axis=0).sort_values('Generic Name')

In [ ]:
# re-index the dataframe
combined_df.reset_index(drop=True, inplace=True)

In [ ]:
# combined_df

### The product dataset

In [ ]:
# investigate the products on columns
pdt_df.columns

In [ ]:
# new_pdt_df= pdt_df.drop(['COUNTRY OF ORIGIN', 'FORMULATION', 'PACK SIZE', 'RETAIL PRICE PER SMALLEST UNIT', 
#              'RETAIL PRICE PER SMALLEST UNIT.1', 'PACK SIZE', 'STRENGTH', 'eshop_id'], axis=1)

In [ ]:
# investigate the product's types
pdt_df.dtypes

In [ ]:
new_dp_list.head()

In [ ]:
# change product's id for ease in comparing with other dataframes
new_pdt_df = pdt_df.rename(columns={'ID': 'PRODUCT ID'})


In [ ]:
# check to find out whether the new_product_df id is identical to the duplicate list id 
merged_pdt_df = new_pdt_df['PRODUCT ID'].isin(new_dp_list['PRODUCT ID'])

In [ ]:
# drop the identical ids
new_pdt_df.drop(new_pdt_df[merged_pdt_df].index, inplace=True)

In [ ]:
# checkout the new dataframe
new_pdt_df.info()

In [ ]:
# test on a product from the duplicates list
new_pdt_df[new_pdt_df['PRODUCT ID'] == 3951]

In [ ]:
# merged_pdt_df[merged_pdt_df['PRODUCT ID'] == 3951]

### The batches dataframe

In [ ]:
batches_df.info()

In [ ]:
# cleanup data by renaming pandas series with more meaningfull names
batches_df = batches_df.rename(columns={'product__generic_name': 'Generic Name', 'product__brand_name': 'Brand Name'})

In [ ]:
new_dp_list.info()

In [ ]:
# create a new column on which we will compare our product ids
batches_df['NEW ID'] = ''

# create a new column on which we will compare our generic names
batches_df['new_generic_name'] = ''

# loop through the unique products dataframe using its index
for i in range(len(unique_pdts_df)):
    
   # loop through the list of duplicates using its index
    for j in range(len(new_dp_list)):
        generic_unique = unique_pdts_df['GENERIC NAME'][i].lower()
        generic_batch = batches_df['Generic Name'][j].lower()
        
       # compare the two pandas series on generic names. Coerce them to lowercase for proper comparison
        if generic_unique in generic_batch:
            
            # assign the new ids to the resulting product from unique products loop and do the same for generic name
            batches_df['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]
            batches_df['new_generic_name'][j] = unique_pdts_df['GENERIC NAME'][i]  

In [ ]:
batches_df.head()

In [ ]:
# subset the rows that were filled
filled_batches = batches_df[batches_df['NEW ID'] != '']

filled_batches

In [ ]:
# subset the rows that were not filled
unfilled_batches = batches_df[batches_df['NEW ID'] == '']

unfilled_batches

In [ ]:
unfilled_batches.reset_index(drop=True, inplace=True)

In [ ]:
# loop through the unique products dataframe using its index
for i in range(len(unique_pdts_df)):
    
    # loop through the list of duplicates using its index
    for j in range(len(unfilled_batches)):
        
        # extract string from the product
        str_a  = unique_pdts_df['GENERIC NAME'][i]
        str_b = unfilled_batches['Generic Name'][j]
        
        # assign ratio to both strings
        ratio = fuzz.partial_ratio(str_b, str_a)
     
        # sent conditon for comparing the series
        if ratio >= 100:
            print(str_a, ':', str_b, ':', ratio)            
            unfilled_batches['new_generic_name'][j] = unique_pdts_df['GENERIC NAME'][i]
            unfilled_batches['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]

In [ ]:
# loop through the unique products dataframe using its index
for i in range(len(unique_pdts_df)):
    
    # loop through the list of duplicates using its index
    for j in range(len(unfilled_batches)):
        
        # extract string from the product
        str_a  = unique_pdts_df['GENERIC NAME'][i]
        str_b = unfilled_batches['Generic Name'][j]
        unique_brand = unique_pdts_df['BRAND NAME'][i].lower()
        batch_brand = batches_df['Brand Name'][j].lower()
        
        # assign ratio to both strings
        first_ratio = fuzz.partial_ratio(str_b, str_a)
        second_ratio = fuzz.partial_ratio(batch_brand, unique_brand)
     
        # sent conditon for comparing the series
        if first_ratio >= 80 and second_ratio >= 80:
            print(str_a, ':', str_b, ':', first_ratio, second_ratio)            
            unfilled_batches['new_generic_name'][j] = unique_pdts_df['GENERIC NAME'][i]
            unfilled_batches['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]

In [ ]:
unfilled_batches.head()

In [ ]:
unfilled_batches = unfilled_batches[unfilled_batches['NEW ID'] != '']
unfilled_batches

In [ ]:
# recipe_df.columns

In [ ]:
new_recipe_df = recipe_df.drop(['id', 'created', 'modified', 'prescription', 'dosage', 'tablets', 'quantity', 'frequency', 
                                'duration', 'route', 'price_discount', 'serving_pharmacy', 'price_controller', 'rx_price',
                               'external_price', 'requires_approval', 'approved', 'dropped', 'canceling_party', 
                                'reason_for_cancelation', 'authorized_by_tmcg', 'authorized_by_insurer', 'client_decision', 
                                'insurer_code'], axis=1)

In [ ]:
# new_recipe_df.info()

In [ ]:
# new_recipe_df.head()

In [ ]:
dp_list.head()

In [ ]:
# unique_pdts_df.info()

In [ ]:
# dp_list.columns

In [ ]:
# dp_list.info()

In [ ]:
new_dp_list.head()

In [ ]:
new_dp_list.info()

In [ ]:
# unique_pdts_df.head(49)

In [ ]:
# new_dp_list['PRODUCT ID'].to_list()

In [ ]:
new_dp_list.tail()

In [ ]:
new_dp_list.tail()

In [ ]:
new_dp_list.info()

In [ ]:
# new_dp_list['PRODUCT ID'].to_list()

In [ ]:
new_pdt_df = new_pdt_df.rename(columns={'ID': 'PRODUCT ID'})

In [ ]:
new_pdt_df.head()

In [ ]:
merged_pdt_dup = pd.merge(new_dp_list, new_pdt_df, on=['PRODUCT ID'], how='inner')

In [ ]:
# merged_pdt_dup.head(51)

In [ ]:
merged_batch_dup = pd.merge(new_dp_list, new_batches_df, on=['PRODUCT ID'], how='inner')

In [ ]:
merged_batch_dup.info()

In [ ]:
new_dp_list.shape

In [ ]:
# new_recipe_df

In [ ]:
new_recipe_df = new_recipe_df.rename(columns={'product_id': 'PRODUCT ID'})

In [ ]:
new_recipe_df.head()

In [ ]:
merged_recipe_dup = pd.merge(new_dp_list, new_recipe_df, on=['PRODUCT ID'], how='inner')

In [ ]:
merged_recipe_dup.info()

In [ ]:
# new_pdt_df
# new_batches_df
# new_recipe_df

# merged_pdt_dup
# merged_batch_dup
# merged_recipe_dup

In [ ]:
unique_pdts_df.head()

In [ ]:
merged_batch_dup.head()

In [ ]:
new_batch = merged_batch_dup.replace()

In [ ]:
# for prod in merged_batch_dup['Generic Name']:
#     if np.where(merged_batch_dup['Generic Name'] == unique_pdts_df['GENERIC NAME'], True, False):
#         print(prod)

In [ ]:
merged_series = merged_batch_dup['Generic Name']
unique_series = unique_pdts_df['GENERIC NAME']

In [ ]:
merged_batch_dup.columns

In [ ]:
# merged_row = merged_batch_dup.loc[['Montelukast']]
# merged_row

In [ ]:
merged_batch_dup['Generic Name'].unique()

In [ ]:
unique_pdts_df['GENERIC NAME'].unique()

In [ ]:
unique_pdts_df.columns

In [ ]:
pdt_df.head()

In [ ]:
pdt_df['NEW ID'] = ""

In [ ]:
for i in range(len(unique_pdts_df)):
    for j in range(len(pdt_df)):
#         if unique_pdts_df['GENERIC NAME'][i] == pdt_df['GENERIC NAME'][j]:
#             pdt_df['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]
        if unique_pdts_df['GENERIC NAME'][i].lower() in pdt_df['GENERIC NAME'][j].lower():
            pdt_df['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]

In [ ]:
# unique_pdts_df[unique_pdts_df['GENERIC NAME'] == 'Montelukast']

In [ ]:
pdt_df.head()

In [ ]:
unique_pdts_df.head()

In [ ]:
pdt_df[pdt_df['NEW ID'] != ""]

In [ ]:
list(pdt_df['GENERIC NAME'].unique())

In [ ]:
unique_pdts_df['GENERIC NAME'].unique()

In [ ]:
'Montelukast'in(pdt_df['GENERIC NAME'])

In [ ]:
new_dp_list.head(20)

In [ ]:
new_dp_list.columns

In [ ]:
new_dp_list[new_dp_list['Generic Name'] == 'Diclofenac 12.5mg Suppositories']

In [ ]:
unique_pdts_df[unique_pdts_df['GENERIC NAME'] == 'Diclofenac']

In [ ]:
new_dp_list.head(10)

In [ ]:
print(new_dp_list['Generic Name'])

In [ ]:
new_dp_list.info()

In [ ]:
unique_pdts_df[unique_pdts_df['GENERIC NAME'].str.contains('Loratadine')]

In [ ]:
pattern = "+./'"

In [ ]:
unique_pdts_df.loc['Diclofenac 12.5mg Suppositories', 'GENERIC NAME']

In [ ]:
#         if unique_pdts_df['GENERIC NAME'][i] == pdt_df['GENERIC NAME'][j]:
#             pdt_df['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]
#         elif new_dp_list['Generic Name'][j].lower().contains(pat="+./'", regex=True) in new_dp_list['Generic Name'][j].lower():
#             new_dp_list['NEW ID'][j] = unique_pdts_df['PRODUCT ID'][i]